In [138]:
import pandas as pd
import faker
import re

In [139]:
df_org = pd.read_csv('orgs.csv')
df_org

,org
0,Movimento 5 Stelle
1,Lega
2,Partito Democratico
3,Forza Italia
4,Fratelli d'Italia
...,...
260,"Unione Italiana Lavoratori del Tessile, Energi..."
261,"Unione italiana lavoratori della chimica, ener..."
262,Unione Sindacale Italiana Poliziotti
263,Unità sindacale Falcri Silcea Sinfub


In [200]:
df = pd.read_csv('dahlia.csv')\
    .rename(columns={'CLEAN_TXN_STRING': 'txnDesc'})
df['fdDescType'] = df['fdDescType'].replace([12,9], 1)
df

,txnDesc,fdDescType,fdCode
0,SUMUP *,1,3007
1,LA CAFFERIA,0,0
2,IONOS SE,0,0
3,AMAZON MAR ######,1,30
4,PLAYSTATION NETWORK,0,0
...,...,...,...
991354,IMPIO PASTICCERIA,0,0
991355,ISP 6368AVERSA,0,0
991356,SEPHORA 1047 MARCIANIS,0,0
991357,PAYPAL *,1,3001


In [201]:
df.fdDescType.value_counts()

0    818535
1    172824
Name: fdDescType, dtype: int64

In [202]:
from random import randint
import random
fake = faker.Faker('it_IT')

faking = {
    'address': fake.address,
    'iban': fake.iban,
    'email': fake.email,
    'ip': fake.ipv4,
    'name': fake.name,
    'honorific': lambda: fake.prefix() + ' ' + fake.name(),
    'phone_number': fake.phone_number,
    'credit_card': fake.credit_card_number,
    'alphanumeric': lambda: fake.password(length=randint(5,13), special_chars=False, digits=True, upper_case=True, lower_case=True),
    'numeric': lambda: fake.random_number(digits=randint(6,9)),
}

In [203]:
df['txnDesc'] = df['txnDesc'].apply(lambda x: df_org.sample(1).iloc[0][0] if pd.isna(x) else x)
df['txnDesc'] = df['txnDesc'].apply(lambda x: x.lower())
df['txnDesc'] = df['txnDesc'].apply(lambda x: re.sub(r'#+', str(faking['numeric']()), x) if re.search(r'#+', x) else x)
df['txnDesc'] = df['txnDesc'].apply(lambda x: re.sub(r'ttttt+', str(faking['alphanumeric']()), x) if re.search(r'ttttt+', x) else x)
df['txnDesc'] = df.apply(lambda row: row.txnDesc + ' ' + random.choice([str(faking[random.choice(list(faking.keys()))]()),'', '']) if re.search(r'30\d{2}$', row.fdCode) else row.txnDesc, axis=1)
df['txnDesc'] = df.apply(lambda row: str(faking['name']()) if re.search(r'<farmacia>', row.txnDesc) else row.txnDesc, axis=1)
df['txnDesc'] = df.apply(lambda row: str(faking['name']()) if re.search(r'_____', row.txnDesc) else row.txnDesc, axis=1)
df['fdDescType'] = df.apply(lambda row: '0' if re.search(r'[4-9]\d{3}$', row.fdCode) else row.fdDescType, axis=1)
df['fdDescType'] = df.apply(lambda row: '0' if re.search(r'30\d{2}$', row.fdCode) and re.search(r' \* $', row.txnDesc) else row.fdDescType, axis=1)
df['txnDesc'] = df.apply(lambda row: row.txnDesc + ' ' + str(faking['name']()) if re.search(r'^\d{3}$', row.fdCode) else row.txnDesc, axis=1)

In [204]:
df.drop(columns=['fdCode'], inplace=True)
df['fdDescType'] = df.fdDescType.astype(int)
df = df.rename(columns={'fdDescType': 'target'})

In [205]:
new_data = [{'txnDesc': fake.company() + ' ' + str(faking[random.choice(list(faking.keys()))]()), 'target': 1} for i in range(100000)]
new_data.extend([{'txnDesc': str(faking[random.choice(list(faking.keys()))]()), 'target': 1} for i in range(100000)])
df = df.append(pd.DataFrame(new_data), ignore_index=True)

In [206]:
df.target.value_counts()

0    873530
1    317829
Name: target, dtype: int64

In [208]:
from sklearn.utils import resample

n_samples = 350000

dfs = [df[df.target == value] for value in df.target.unique()]

dfs_resampled = [resample(df, replace=True, n_samples=n_samples) for df in dfs]

df = pd.concat(dfs_resampled)

In [209]:
df.target.value_counts()

0    350000
1    350000
Name: target, dtype: int64

In [210]:
df.to_csv('dataset.csv', index=False)

In [211]:
df = pd.read_csv('dataset.csv')

In [223]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

# Download the Italian stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('italian'))

def process_text(text):

    text = text.lower()
    
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)

    text = re.sub('\*+', ' ', text)
    
    # Tokenize the text
    words = text.split()
    
    # Remove the stop words
    words = [word for word in words if word not in stop_words]
    
    return ' '.join(words)

df['processed_text'] = df.apply(lambda row: process_text(row.txnDesc), axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mohamed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [224]:
df

,txnDesc,target,processed_text
0,paypal *,0,paypal
1,il ristorante pizzeria,0,ristorante pizzeria
2,windtre ricarica onlin,0,windtre ricarica onlin
3,scalapay ritmo shoes,0,scalapay ritmo shoes
4,mc donalds,0,mc donalds
...,...,...,...
699995,zurich insurance company ltd 13796384 sdd a...,1,zurich insurance company ltd 13796384 sdd addd...
699996,Moschino-Modigliani Group Fabrizia Acerbi,1,moschino-modigliani group fabrizia acerbi
699997,zolivetti@example.net,1,zolivetti@example.net
699998,Lippomano-Paganini SPA 3542868720122562,1,lippomano-paganini spa 3542868720122562


In [225]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the words back into a string
# df['processed_text'] = df['processed_text'].apply(' '.join)

# Create the transform
vectorizer = TfidfVectorizer(ngram_range=(2, 3), max_features=10000)

# Tokenize and build vocab
vectorizer.fit(df['processed_text'])

# Encode document
vector = vectorizer.transform(df['processed_text'])

# Summarize encoded vector
print(vector.shape)

(700000, 10000)


In [226]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.metrics import f1_score
#from cuml import SVC

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(vector, df['target'], test_size=0.3, random_state=42)

# Define the SVM model
clf = svm.SVC()

# Define the parameter grid for GridSearchCV
# param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear']} 
param_grid = {'C': [0.1], 'gamma': [1], 'kernel': ['linear']} 

# Use GridSearchCV for hyperparameter tuning
grid = GridSearchCV(clf, param_grid, refit=True, verbose=4, n_jobs=-1)
grid.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = grid.predict(X_test)

# Evaluate the model using the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
F1 Score: 0.7459872587541823


In [227]:
from sklearn.metrics import recall_score

recall = recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall}')

Recall: 0.7603714285714286


In [228]:
from joblib import dump

# Save the model to a pickle file
dump(grid, 'model.pkl')
dump(vectorizer, 'tfidf.pkl')
# dump(svd, 'svd.pkl')

['tfidf.pkl']

In [134]:
from joblib import load
model = load('model.pkl')
tfidf = load('tfidf.pkl')

In [240]:
txt = 'netflix d96h1ihO7'
txt = process_text(txt)
txt = tfidf.transform([txt])
model.predict(txt)

array([0], dtype=int64)